In [14]:
from collections import namedtuple
import pickle
import pandas as pd
from zillow_adapter import ask_zillow
from sklearn.ensemble import RandomForestRegressor


with open("final_df.pickle", "rb") as fp: 
    X_, y = pickle.load(fp)
    
def prep(dat: pd.DataFrame) -> pd.DataFrame: 
    
    return (dat.drop(['zip5_encoded', 'city_encoded'], axis=1)
            .rename(columns={'year_built_bin': 'year_built', 'baths_lavs': 'bathrooms', 'beds_total': 'bedrooms'})
            .assign(**{feat: pd.cut(dat[feat], 
                                    bins=[0, 4, 9, 14, 10000], 
                                    labels=["0-4 years", "5-9 years", "10-14 years", "15+"]) 
                       for feat 
                       in ['roof_year', 'furnace_year']}))

X = prep(X_)

address, zipcode = "4565 White Rd, Pierson, MI", 49339
house = ask_zillow(address, zipcode)

print(X.shape)
inps = {feat: house.results.__dict__[feat] for feat in ('bedrooms', 'bathrooms', 'year_built')}

countertops_map = {"Marble/Quartz": [1, 0, 0, 0],
                   "Granite/Concrete": [0, 1, 0, 0],
                   '"Formica/Tile"': [0, 0, 1, 1], 
                   'Laminate': [0, 0, 0, 0], 
                   'skip': [X.quartz_countertops.mean(), 
                            X.granite_countertops.mean(), 
                            X.formica_countertops.mean(), 
                            X.tile_countertops.mean()]}

flooring_map = {"Hardwood" : [0, 1], 
                "Engineered/Laminate": [1, 0], 
                "Ceramic Tile": [0, 0], 
                "Porcelain Tile/Concrete": [0, 0], 
                "skip": [X.laminate_floors.mean(), 
                         X.hardwood_floors.mean()]}

roof_furnace_map = {**{'skip': y.mean()}, **{feat: feat 
                                             for feat 
                                             in ["0-4 years", "5-9 years", "10-14 years", "15+"]}}

X.head()

(24773, 11)


,bedrooms,roof_year,bathrooms,year_built,furnace_year,quartz_countertops,granite_countertops,formica_countertops,tile_countertops,laminate_floors,hardwood_floors
0,4,15+,2.1,1999,15+,0,1,0,0,0,0
3,5,15+,3.0,1996,15+,0,0,0,0,0,1
6,3,15+,2.1,1997,15+,0,1,0,0,0,0
10,4,15+,3.0,1992,15+,0,1,0,0,0,1
11,4,15+,3.1,1980,15+,0,0,0,0,0,1


In [15]:
roof_furnace_map

{'skip': 153.82237057709239,
 '0-4 years': '0-4 years',
 '5-9 years': '5-9 years',
 '10-14 years': '10-14 years',
 '15+': '15+'}

In [47]:
{feat: house.results.__dict__[feat] for feat in ('bedrooms', 'bathrooms', 'year_built')}

{'bedrooms': '3', 'bathrooms': '1.0', 'year_built': '1980'}

In [ ]:

# dog = pre.copy()

# dog.furnace_year = dog.furnace_year.replace({2019:0})

# bins = [0,5,10,15,2000]
# labels = ['0 to 4', '5 to 9', '10 to 14', '15+']

# dog['furnace_year_binned'] = pd.cut(dog['furnace_year'], 
#                                     bins, 
#                                     labels = labels, 
#                                     right=False,
#                                     include_lowest=True)

# dog['roof_year_binned'] = pd.cut(dog['roof_year'], 
#                                     bins, 
#                                     labels = labels, 
#                                     right=False,
#                                     include_lowest=True)

#from zillow_adapter import X

# zill_props = ['bathrooms',
#  'bedrooms',
#  'data',
#  'get_attr',
#  'graph_data_link',
#  'home_detail_link',
#  'home_size',
#  'home_type',
#  'last_sold_date',
#  'last_sold_price',
#  'last_sold_price_currency',
#  'latitude',
#  'longitude',
#  'map_this_home_link',
#  'property_size',
#  'tax_value',
#  'tax_year',
#  'year_built',]

# user_props = ['furnance_year', 'roof_year', 'countertops', 'kitchen_floor']

# X.results.__dict__

In [14]:
Home = namedtuple("Home", zill_props + ['countertops', 'roof_age'])

#Home([X.results.__dict__[feat] for feat in zill_props])

KeyError: 'get_attr'

In [3]:
help(namedtuple)

Help on function namedtuple in module collections:

namedtuple(typename, field_names, *, verbose=False, rename=False, module=None)
    Returns a new subclass of tuple with named fields.
    
    >>> Point = namedtuple('Point', ['x', 'y'])
    >>> Point.__doc__                   # docstring for the new class
    'Point(x, y)'
    >>> p = Point(11, y=22)             # instantiate with positional args or keywords
    >>> p[0] + p[1]                     # indexable like a plain tuple
    33
    >>> x, y = p                        # unpack like a regular tuple
    >>> x, y
    (11, 22)
    >>> p.x + p.y                       # fields also accessible by name
    33
    >>> d = p._asdict()                 # convert to a dictionary
    >>> d['x']
    11
    >>> Point(**d)                      # convert from a dictionary
    Point(x=11, y=22)
    >>> p._replace(x=100)               # _replace() is like str.replace() but targets named fields
    Point(x=100, y=22)



In [3]:
from zillow_adapter import X

X.results

In [7]:
from zillow_adapter import ask_zillow

print(ask_zillow("fish", 10223).results)


None
